In [441]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats
from dash import Dash, html, dcc, Input, Output
from sklearn.linear_model import LinearRegression

1.	wstępna analiza danych i czyszczenie:
a.	duplikaty
b.	wartości odstające
c.	spójność
d.	braki w danych
e.	skala wartości
f.	inne
2.	wizualizacja rozkładu zmiennych, zależności ceny od innych zmiennych, liczebność kategorii
3.	budowa modelu regresji ceny od pozostałych zmiennych. Istotne zmienne należy wybrać eliminacją wsteczną lub selekcją postępującą.
4.	wizualizacja modelu regresji
5.	stworzenie dashboardu z powyższymi wizualizacjami i próbką danych w postaci tabeli. Dashboard powinien umożliwiać zmianę parametrów wykresów - np. zależność ceny od innej, wybranej przez użytkownika zmiennej

Wszystkie wizualizacje i dane powinny zostać zaprezentowane w postaci dashboardu. Etapy czyszczenia danych i budowy modelu regresji należy zawrzeć w notatniku Jupytera (.ipynb).


In [442]:
#porządkowanie tabeli
df = pd.read_csv("messy_data.csv", names=['carat', 'clarity' , 'color' ,'cut', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price'])
df.drop(index=0, inplace=True)

In [443]:
#odrzucenie duplikatów
df.drop_duplicates(subset=None, inplace=True)

In [444]:
#zamiana dziwnych i pustych wartości na NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [445]:
#konwersja na typ float
df['carat'] = df['carat'].astype(float)
df['x dimension'] = df['x dimension'].astype(float)
df['y dimension'] = df['y dimension'].astype(float)
df['z dimension'] = df['z dimension'].astype(float)
df['depth'] = df['depth'].astype(float)
df['table'] = df['table'].astype(float)
df['price'] = df['price'].astype(float)

#zamiana na duże litery
df['cut'] = df['cut'].str.upper()
df['clarity'] = df['clarity'].str.upper()
df['color'] = df['color'].str.upper()

In [446]:
#zamiana NaN na średnią tam gdzie się da
df['carat'].fillna(df["carat"].mean(), inplace=True)
df['x dimension'].fillna(df["x dimension"].mean(), inplace=True)
df['y dimension'].fillna(df["y dimension"].mean(), inplace=True)
df['z dimension'].fillna(df["z dimension"].mean(), inplace=True)
df['depth'].fillna(df["depth"].mean(), inplace=True)
df['table'].fillna(df["table"].mean(), inplace=True)
df['price'].fillna(df["price"].mean(), inplace=True)

#i usunięcie tam gdzie się nie da
df.dropna(inplace=True)

In [447]:
# Usuwanie wartości odstających
numeric_columns = ['carat', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price']

z = np.abs(stats.zscore(df['carat']))

threshold_z = 2
outlier_indices = np.where(z > threshold_z)[0]
no_outliers = df.drop(outlier_indices)
df = no_outliers

In [448]:

# Enkodowanie danych
def encode_categorical(df, column):
    unique_values = df[column].unique()
    encoding = {val: i for i, val in enumerate(unique_values)}
    df[column] = df[column].map(encoding)
    return df

def column_contains_strings(df, column_name):
    return df[column_name].dtype == 'object'

In [449]:
colors = {
    'background' :'#ADD8E6',
    'text': '7FDBFF'
}

In [450]:
#Dashboard
app = Dash(__name__)

app.layout = html.Div(style={'backgroundColor': colors['background']},
                      children=[

                        html.H1(
                              children="Projekt PAD",
                              style={
                                  'text align': 'center',
                                  'color' : colors['text']
                              }
                          ),
                        html.Div(children="Interaktywny dashboard",
                                   style={
                                  'text align': 'center',
                                  'color' : colors['text']
                              }),
                        dcc.Dropdown(['carat', 'clarity' , 'color' ,'cut', 'x dimension',
                                                     'y dimension', 'z dimension', 'depth', 'table', 'price'],'carat', id='xdropdown'),
                                      
                                            
            dcc.Graph(
                id='graph'
            )])

@app.callback(
    Output('graph', 'figure'),
    Input('xdropdown', 'value'),
)
def update_graph(x_column):
    fig = px.scatter(df, x=x_column, y='price')
    
    if column_contains_strings(df, x_column):
        df_encoded = encode_categorical(df.copy(), x_column)
    else:
        df_encoded = df
        
    # Regresja liniowa
    X = df_encoded[x_column].values.reshape(-1, 1)
    y = df_encoded['price']
    model = LinearRegression()
    model.fit(X, y)
    x_range = np.array([X.min(), X.max()])
    y_pred = model.predict(x_range.reshape(-1, 1))

    # Dodanie regresji do plota
    fig.add_scatter(x=x_range.flatten(), y=y_pred, mode='lines', name='Linear Regression')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)